# Query Planning in Llama-Index

Query planning is a useful tool to leverage the ability of LLMs to structure the user inputs into multiple different queries, either sequentially or in parallel before answering the questions. This method improvers the response by allowing the question to be decomposed into smaller, more answerable questions.

Sub-question queries are one such method. Sub-question queries decompose the user input into multiple different sub-questions. This is great for answering complex questions that require knowledge from different documents.

Relatedly, there are a great deal of configurations for this style of application that must be selected. In this example, we'll iterate through several of these choices and evaluate each with TruLens.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/frameworks/llama_index/llama_index_queryplanning.ipynb)

### Import from Llama-Index and TruLens

In [2]:
#! pip install trulens_eval==0.7.0 llama_index==0.7.11

In [1]:
from llama_index import VectorStoreIndex, SimpleWebPageReader, ServiceContext
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()

In [3]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

### Set keys

For this example we need an OpenAI key

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "..."

### Set up evaluation

Here we'll use agreement with GPT-4 as our evaluation metric.

In [5]:
openai = feedback.OpenAI()
model_agreement = Feedback(openai.model_agreement).on_input_output()

✅ In model_agreement, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In model_agreement, input response will be set to *.__record__.main_output or `Select.RecordOutput` .


### Run the dashboard

By starting the dashboard ahead of time, we can watch as the evaluations get logged. This is especially useful for longer-running applications.

In [6]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8507 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [7]:
# load data
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://www.gutenberg.org/files/11/11-h/11-h.htm"]
)

# iterate through embeddings and chunk sizes, evaluating each response's agreement with chatgpt using TruLens
embeddings = ['text-embedding-ada-001','text-embedding-ada-002']
query_engine_types = ['VectorStoreIndex','SubQuestionQueryEngine']

service_context=512

for embedding in(embeddings):
    for query_engine_type in query_engine_types:

            # build index and query engine
            index = VectorStoreIndex.from_documents(documents)

            # create embedding-based query engine from index
            query_engine = index.as_query_engine(embed_model=embedding)

            if query_engine_type == 'SubQuestionQueryEngine':
                service_context = ServiceContext.from_defaults(chunk_size=512)
                # setup base query engine as tool
                query_engine_tools = [
                    QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(name='Alice in Wonderland', description='THE MILLENNIUM FULCRUM EDITION 3.0')
                    )
                ]
                query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools, service_context=service_context)
            else:
                pass         

            tc = TruLlama(app_id = f'{query_engine_type}_{embedding}', app = query_engine, feedbacks = [model_agreement])

            response = tc.query("Describe Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts?")
            response = tc.query("Relate aspects of enchantment to the nostalgia that Alice experiences in Wonderland. Why is Alice both fascinated and frustrated by her encounters below-ground?")
            response = tc.query("Describe the White Rabbit's function in Alice.")
            response = tc.query("Describe some of the ways that Carroll achieves humor at Alice's expense.")
            response = tc.query("Compare the Duchess' lullaby to the 'You Are Old, Father William' verse")
            response = tc.query("Compare the sentiment of the Mouse's long tale, the Mock Turtle's story and the Lobster-Quadrille.")
            response = tc.query("Summarize the role of the mad hatter in Alice's journey")
            response = tc.query("How does the Mad Hatter influence the arc of the story throughout?")


✅ app VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
✅ feedback def. feedback_definition_hash_4c65c970e87d77a632b18a19706b1f9c -> default.sqlite


model_agreement has been deprecated. Use GroundTruthAgreement(ground_truth) instead.


✅ record record_hash_e553aa0c2f2eea8e2de53fca088939f4 from VectorStoreIndex_text-embedding-ada-001 -> default.sqlite


model_agreement has been deprecated. Use GroundTruthAgreement(ground_truth) instead.


✅ record record_hash_ca8a027e19e194a966c03131b3cddeb6 from VectorStoreIndex_text-embedding-ada-001 -> default.sqlite


...

✅ record record_hash_6b12a569cfba912b744e5846b288b462 from SubQuestionQueryEngine_text-embedding-ada-002 -> default.sqlite
Generated 1 sub questions.
[Alice in Wonderland] Q: What is the role of the mad hatter in Alice's journey?
✅ feedback feedback_result_hash_dca05fb8dc839fe62f98a82c72f741dd on record_hash_6b12a569cfba912b744e5846b288b462 -> default.sqlite
[Alice in Wonderland] A: 
The Mad Hatter plays an important role in Alice's journey. He is a source of advice and guidance, offering Alice hints and tips on how to navigate her way through the strange world she finds herself in. He also provides comic relief, with his nonsensical conversations and his tendency to get into arguments with the March Hare. Finally, he serves as a reminder of the importance of keeping good terms with Time, as Alice learns from his story of how he quarrelled with Time and was unable to get him to do anything he asked.


model_agreement has been deprecated. Use GroundTruthAgreement(ground_truth) instead.


✅ record record_hash_56fcc8e1fb34512b565a718b021252c5 from SubQuestionQueryEngine_text-embedding-ada-002 -> default.sqlite
Generated 3 sub questions.
[Alice in Wonderland] Q: What is the Mad Hatter's role in Alice in Wonderland
✅ feedback feedback_result_hash_f400a1c7840b1294644381a116f00cf8 on record_hash_56fcc8e1fb34512b565a718b021252c5 -> default.sqlite
[Alice in Wonderland] Q: How does the Mad Hatter's role change throughout the story
[Alice in Wonderland] Q: How does the Mad Hatter influence the arc of the story
[Alice in Wonderland] A: 
The Mad Hatter influences the arc of the story by providing comic relief and introducing Alice to the concept of time manipulation. He also encourages Alice to tell a story, which leads to the Dormouse telling a story. This helps to move the plot forward and keep the reader engaged.
[Alice in Wonderland] A: 
The Mad Hatter is a character in Alice in Wonderland who hosts a tea party with the March Hare and the Dormouse. He is known for his nonsensi

model_agreement has been deprecated. Use GroundTruthAgreement(ground_truth) instead.


✅ record record_hash_ce30a14299554db335080f8ae3e14e13 from SubQuestionQueryEngine_text-embedding-ada-002 -> default.sqlite
✅ feedback feedback_result_hash_957bcc94b0e7ac248f1b5dc8611b242a on record_hash_ce30a14299554db335080f8ae3e14e13 -> default.sqlite
